In [69]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import numpy
from torch.optim import Adam, SGD
from torch.utils.data import TensorDataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import chess
import chess.engine
import random
import numpy

In [70]:
#design model 
class ChessCNN(nn.Module):
  def __init__(self):
    super(ChessCNN, self).__init__()
    self.conv1 = nn.Conv2d(14, 32, kernel_size=3, padding=1)
    self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
    self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
    self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
    self.fc1 = nn.Linear(256 * 8 * 8, 1024)
    self.fc2 = nn.Linear(1024, 8 * 8)

  def forward(self, x):
    x = torch.relu(self.conv1(x))
    x = torch.relu(self.conv2(x))
    x = torch.relu(self.conv3(x))
    x = torch.relu(self.conv4(x))
    x = x.view(-1, 256 * 8 * 8)
    x = torch.relu(self.fc1(x))
    x = self.fc2(x)
    #output = torch.softmax(x)
    return x.view(-1, 8, 8)
    #return output

In [71]:
# Create an instance of the model
model = ChessCNN()

# Define a loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [72]:
data = numpy.load("fishybear2_black.npz", allow_pickle=True)
positions, scores = [data[key] for key in data]

tensor_x = torch.Tensor(positions)
tensor_y = torch.Tensor(scores)

data = TensorDataset(tensor_x, tensor_y)

train_set, val_set = random_split(data, lengths = [int(len(scores) * 0.85), len(scores) - int(len(scores) * 0.85)], 
    generator = torch.Generator().manual_seed(2024))

b_size = 64

train_loader = DataLoader(train_set, batch_size = b_size, shuffle = True, num_workers = 2)

val_loader = DataLoader(val_set, batch_size = b_size, shuffle = False, num_workers = 2)

In [73]:
def train_one_epoch():
    running_loss = 0.0
    last_loss = 0.0

    for i, data in enumerate(train_loader):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = model(inputs)


        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000
            running_loss = 0.

    return last_loss

In [74]:
epoch_number = 0

EPOCHS = 7

best_vloss = 1_000_000

In [75]:
for epoch in range(EPOCHS):
    print("EPOCH {}:".format(epoch_number + 1))

    model.train(True) #sets to training mode
    avg_loss = train_one_epoch()

    model.train(False) #sets to evaluation mode

    running_vloss = 0.0
    for i, vdata in enumerate(val_loader):
        vinputs, vlabels = vdata
        voutputs = model(vinputs)
        vloss = criterion(voutputs, vlabels)
        running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print("LOSS train {} valid {}".format(avg_loss, avg_vloss))

    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        torch.save(model.state_dict(), "best_model2.h5")

    epoch_number += 1

EPOCH 1:


LOSS train 0.0 valid 0.013642985373735428
EPOCH 2:
LOSS train 0.0 valid 0.013032774440944195
EPOCH 3:
LOSS train 0.0 valid 0.012853332795202732
EPOCH 4:
LOSS train 0.0 valid 0.012511886656284332
EPOCH 5:
LOSS train 0.0 valid 0.012423526495695114
EPOCH 6:
LOSS train 0.0 valid 0.012328531593084335
EPOCH 7:
LOSS train 0.0 valid 0.012485136277973652
